In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import os
sns.set()

from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

import statsmodels.api as sm

In [2]:
path = '../Pickles/'

In [3]:
car_avg_df = pd.read_pickle(path+'car_avg_stats_df.pkl')

In [4]:
def calc_passer_rating(row):
    comp_p = (row['Cmp'] / row['Att'] - .3) * 5
    pyd_p = (row['Pass_Yds'] / row['Att'] - 3) * .25
    td_p = (row['TD'] / row['Att']) * 20
    int_p = 2.375 - (row['Int'] / row['Att']) * 25
    return sum([comp_p, pyd_p, td_p, int_p]) / 6



In [5]:
def add_features(car_avg_df):
    '''
    Add TD%, Int%, and place QBs in different bins
    arg:
        DF created from PFR_scrape.py
    return:
        Two DF: 1) DF with added stats; 2) DF of 2018 QBs who did not play full 2018 season
    '''
    car_avg_df['YTD_Rating'] = car_avg_df.apply(calc_passer_rating, axis=1)
    
    car_avg_df['TD%'] = car_avg_df.apply(lambda x: x['TD'] / x['Att'], axis=1)
    car_avg_df['Int%'] = car_avg_df.apply(lambda x: x['Int'] / x['Att'], axis=1)
    career_ratings = car_avg_df.groupby('name', as_index=False)['YTD_Rating'].mean().reset_index()

    # binning QBs by tier
    career_ratings['tier'] = pd.qcut(career_ratings['YTD_Rating'], 3, labels=[3, 2, 1])
    career_ratings['tier'] = career_ratings['tier'].astype(int)
    data_stats = career_ratings.merge(car_avg_df, on='name')
    data_stats.rename({'YTD_Rating_x': 'Car Rating', 'YTD_Rating_y': 'YTD_Rating'}, axis=1, inplace=True)

    # saving 2018 injured/benched players for inclusion in model
    benched_injured_df = data_stats[(data_stats['FY_TD'].isna()) &
                                    (data_stats['Year'] == 2018)]
    data_stats.dropna(inplace=True)
    return data_stats, benched_injured_df

In [6]:
data_stats, benched_injured_df= add_features(car_avg_df)

In [7]:
benched_injured_df

,index,name,Car Rating,tier,Year,G,GS,Win,TD,Cmp,...,Att/gm,TD/gm,Pass_Yds/gm,Int/gm,Sk/gm,Career W %,Yrs Xp,YTD_Rating,TD%,Int%
21,3,Andrew Luck,0.848839,2,2018,79,79,51,156,1838,...,37.936709,1.974684,275.822785,0.898734,2.012658,0.645570,7,0.909639,0.052052,0.023690
45,6,Ben Roethlisberger,0.932079,1,2018,216,214,144,363,4616,...,33.185185,1.680556,260.157407,0.879630,2.319444,0.672897,15,0.942487,0.050642,0.026507
51,8,Blake Bortles,0.780913,2,2018,75,73,24,103,1561,...,35.093333,1.373333,235.280000,1.000000,2.600000,0.328767,5,0.806136,0.039134,0.028495
81,13,Cam Newton,0.859849,1,2018,123,122,68,182,2321,...,31.634146,1.479675,231.455285,0.869919,2.317073,0.557377,8,0.864115,0.046775,0.027499
96,16,Case Keenum,0.936086,1,2018,31,30,17,40,690,...,34.419355,1.290323,239.903226,0.709677,1.806452,0.566667,2,0.889195,0.037488,0.020619
187,33,Eli Manning,0.814263,2,2018,223,223,115,354,4709,...,34.865471,1.587444,246.358744,1.031390,1.762332,0.515695,14,0.848475,0.045531,0.029582
281,54,Josh Rosen,0.667356,3,2018,14,13,3,11,217,...,28.071429,0.785714,162.714286,1.000000,3.214286,0.230769,1,0.667356,0.027990,0.035623
310,59,Marcus Mariota,0.908043,1,2018,56,55,27,69,1015,...,28.660714,1.232143,214.357143,0.750000,2.321429,0.490909,4,0.893731,0.042991,0.026168
351,66,Matthew Stafford,0.897102,1,2018,128,128,63,218,3114,...,38.531250,1.703125,279.093750,0.843750,2.351562,0.492188,8,0.904890,0.044201,0.021898
439,85,Ryan Tannehill,0.829884,2,2018,88,88,42,123,1829,...,33.079545,1.397727,232.204545,0.852273,2.818182,0.477273,7,0.870398,0.042254,0.025764


In [8]:
data_stats.head()

,index,name,Car Rating,tier,Year,G,GS,Win,TD,Cmp,...,Att/gm,TD/gm,Pass_Yds/gm,Int/gm,Sk/gm,Career W %,Yrs Xp,YTD_Rating,TD%,Int%
0,0,Aaron Rodgers,1.02425,1,2008,16,16,6,28,341,...,33.500000,1.750000,252.375000,0.812500,2.125000,0.375000,1,0.937966,0.052239,0.024254
1,0,Aaron Rodgers,1.02425,1,2009,32,32,17,58,691,...,33.656250,1.812500,264.750000,0.625000,2.625000,0.531250,2,0.985395,0.053853,0.018570
2,0,Aaron Rodgers,1.02425,1,2010,47,47,27,86,1003,...,33.021277,1.829787,263.702128,0.659574,2.446809,0.574468,3,0.993610,0.055412,0.019974
3,0,Aaron Rodgers,1.02425,1,2011,62,62,41,131,1346,...,33.129032,2.112903,274.790323,0.596774,2.435484,0.661290,4,1.050065,0.063778,0.018014
4,0,Aaron Rodgers,1.02425,1,2012,78,78,52,170,1717,...,33.410256,2.179487,273.487179,0.576923,2.589744,0.666667,5,1.056456,0.065234,0.017268


In [ ]:
car_avg_df['TD%'] = car_avg_df.apply(lambda x: x['TD'] / x['Att'], axis=1)

In [ ]:
car_avg_df['Int%'] = car_avg_df.apply(lambda x: x['Int'] / x['Att'], axis=1)

In [ ]:
car_avg_df.describe(include='all')

In [ ]:
career_ratings = car_avg_df.groupby('name', as_index=False).agg({'YTD_Rating':'mean'})
career_ratings

In [ ]:
career_ratings.describe()

In [ ]:
plt.hist(career_ratings['YTD_Rating'])

## Used qcut to classify QBs into 3 different tiers

In [ ]:
career_ratings['tier'] = pd.qcut(career_ratings['YTD_Rating'], 3, labels = [3,2,1])

In [ ]:
career_ratings['tier'] = career_ratings['tier'].astype(int)

In [ ]:
car_avg_df.columns

In [ ]:
data_stats = career_ratings.merge(car_avg_df, on='name')

In [ ]:
data_stats.rename({'YTD_Rating_x':'Car Rating', 'YTD_Rating_y':'YTD_Rating'}, axis=1, inplace=True)

In [ ]:
# A few key 2018 QBs got hurt. Stashing their data here before I drop all NaNs
benched_injured_df = data_stats[(data_stats['FY_TD'].isna()) &
          (data_stats['Year'] == 2018)]

In [ ]:
data_stats.dropna(inplace=True)

In [ ]:
data_stats.info()

In [ ]:
car_avg_stats = ['Year','FY_TD','G','Career W %','Cmp/gm',
                 'Att/gm','TD/gm','TD%','Int%','Pass_Yds/gm','Int/gm',
                 'Sk/gm', 'Yrs Xp', 'tier']

In [ ]:
car_avg_graph_df = data_stats[car_avg_stats]
car_avg_graph_df.info()

## QB tier analysis

In [ ]:
tier1_graph_df = car_avg_graph_df[car_avg_graph_df['tier'] == 1]
tier2_graph_df =car_avg_graph_df[car_avg_graph_df['tier'] == 2]
tier3_graph_df =car_avg_graph_df[car_avg_graph_df['tier'] == 3]

In [ ]:
tier1_graph_df.describe(include='all')

In [ ]:
tier2_graph_df.describe(include='all')

In [ ]:
tier3_graph_df.describe(include='all')

In [ ]:
fig = plt.figure(figsize=(14,10))
ax1 = sns.heatmap(tier1_graph_df.corr(), cmap='seismic', annot=True, vmin=-1, vmax=1)

In [ ]:
fig = plt.figure(figsize=(14,10))
ax2 = sns.heatmap(tier2_graph_df.corr(), cmap='seismic', annot=True, vmin=-1, vmax=1)

In [ ]:
fig = plt.figure(figsize=(14,10))
ax3 = sns.heatmap(tier3_graph_df.corr(), cmap='seismic', annot=True, vmin=-1, vmax=1)

In [ ]:
sns.pairplot(car_avg_graph_df, height=1.2, aspect=1.5)

## Test Set thinking
2018 is the test set to compare to 2019 actual performance.


In [ ]:
car_avg_graph_df.head()

In [ ]:
test_df_2018 = data_stats[data_stats['Year'] == 2018]

In [ ]:
train_val_df = car_avg_graph_df[car_avg_graph_df['Year'] < 2018].copy()

# Preliminary feature engineering

In [ ]:
train_val_df.head()

In [ ]:
def add_deviation_feature(X, feature, category):
    
    # temp groupby object
    category_gb = X.groupby(category)[feature]
    
    # create columns of category means and standard deviations
    category_mean = category_gb.transform(lambda x: x.mean())
    category_std = category_gb.transform(lambda x: x.std())
    
    # compute stds from category mean for each feature value,
    # add to X as new feature
    deviation_feature = (X[feature] - category_mean) / category_std 
    X[feature + '_Dev_' + category] = deviation_feature
    X[feature + '_Dev_' + category].fillna(value=0, inplace=True)

In [ ]:
add_deviation_feature(train_val_df, 'TD/gm', 'tier')

# Working with non-test set

## Create model in sm and perform LR Assumption / residual checks

In [ ]:
# log-transformed FY TD
X = train_val_df.drop(['FY_TD'], axis=1)
y = np.log(train_val_df['FY_TD'])

### Note to self, all y has been transformed at this point (including for test set)

In [ ]:
assumption_test_df = train_val_df.copy()

In [ ]:
def stats_model_for_residuals(df, X, y):
    x_for_sm = sm.add_constant(X)
    sm_linear_all = sm.OLS(y, X).fit()
    
    df['predict']=sm_linear_all.predict(X)
    df['resid']= y-df['predict']
    with sns.axes_style('white'):
        plot = df.plot(
            kind='scatter', x='predict', y='resid', alpha=0.5, figsize=(10,6))
    return df
    

In [ ]:
df_with_residuals = stats_model_for_residuals(train_val_df, X, y)

In [ ]:
plt.figure(figsize=(14,10))
stats.probplot(df_with_residuals['resid'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot")
plt.show()

Residuals appear to be slightly left skewed with minor fluctuations in the error variance.

# Linear Regression Modeling

# Cross Validation

In [ ]:
# this is for log transformed only
def MSE_calc(y_val, val_pred):
    return sum((np.exp(y_val) - np.exp(val_pred))**2)/len(y_val)

In [ ]:
def split_and_CV(X, y):
    X, y = np.array(X), np.array(y)
    kf = KFold(n_splits=4, shuffle=True)
    cv_lm_r2 = []
    cv_ridge_r2 = []
    cv_lm_MSE = []
    
    for train_ind, val_ind in kf.split(X,y):
        X_train, y_train = X[train_ind], y[train_ind]
        X_val, y_val = X[val_ind], y[val_ind]
        
        # Linear model
        lr_model = LinearRegression()
        lr_model.fit(X_train, y_train)
        cv_lm_r2.append(lr_model.score(X_val, y_val))
        val_pred = lr_model.predict(X_val)
        MSE_man = MSE_calc(y_val, val_pred)
        cv_lm_MSE.append(MSE_man)

        # Ridge model
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        lm_ridge = Ridge(alpha=.1)
        lm_ridge.fit(X_train_scaled, y_train)
        cv_ridge_r2.append(lm_ridge.score(X_val_scaled, y_val))
    
    print(f'R^2 LM reg: {cv_lm_r2}')
    print(f'LM reg mean cv R^2: {np.mean(cv_lm_r2):.3f} +- {np.std(cv_lm_r2):.3f}')
    print(f'MSE: {cv_lm_MSE}')
    print(f'MSE simple mean: {np.mean(cv_lm_MSE)}\n')

    print(f'R^2 Ridge: {cv_ridge_r2}')
    print(f'LM ridge mean cv R^2: {np.mean(cv_ridge_r2):.3f} +- {np.std(cv_ridge_r2):.3f}\n')

In [ ]:
split_and_CV(X,y)

# Feature Engineering

In [ ]:
# polynomial regression

def poly_add(X,y,alpha=1):
    cv_poly_r2 = []
    cv_poly_MSE = []
    cv_ridge_r2 = []
    cv_ridge_MSE = []
    cv_LASSO_r2 = []
    cv_LASSO_MSE = []
    
    # Polynomial factors
    
    X['PY/G^2'] = X['Pass_Yds/gm'] ** 3 + X['Pass_Yds/gm'] ** 2
    X['Cmp/G^2'] = X['Cmp/gm'] ** 3 + X['Cmp/gm'] ** 2
    X['TD/gm^2'] = X['TD/gm'] ** 3 + X['TD/gm'] ** 2
    X['Yrs Xp^2'] = X['Yrs Xp']**2
    
    # Add interaction terms
    X['TD%_/_Int%'] = X['TD%'] / X['Int%']
    
    
    X, y = np.array(X), np.array(y)   
    
    kf = KFold(n_splits=4, shuffle=True)

    for train_ind, val_ind in kf.split(X,y):
        X_train, y_train = X[train_ind], y[train_ind]
        X_val, y_val = X[val_ind], y[val_ind]
        
        # Feature engineered model
        poly_model = LinearRegression()
        poly_model.fit(X_train, y_train)
        cv_poly_r2.append(poly_model.score(X_val, y_val))
        val_pred = poly_model.predict(X_val)
        MSE_man = MSE_calc(y_val, val_pred)
        cv_poly_MSE.append(MSE_man)
        
        #### Regularization Section ####
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        # Ridge
#         lm_ridge = Ridge(alpha=alpha)
#         lm_ridge.fit(X_train_scaled, y_train)
#         cv_ridge_r2.append(lm_ridge.score(X_val_scaled, y_val))
#         val_pred = lm_ridge.predict(X_val_scaled)
#         MSE_man = MSE_calc(y_val, val_pred)
#         cv_ridge_MSE.append(MSE_man)
        
        # LASSO
        lasso_model = Lasso(alpha=alpha)
        lasso_model.fit(X_train_scaled, y_train)
        cv_LASSO_r2.append(lasso_model.score(X_val_scaled, y_val))
        val_pred = lasso_model.predict(X_val_scaled)
        MSE_man = MSE_calc(y_val, val_pred)
        cv_LASSO_MSE.append(MSE_man)
        
    print(f'R^2 Poly: {cv_poly_r2}')
    print(f'Poly mean cv R^2: {np.mean(cv_poly_r2):.3f} +- {np.std(cv_poly_r2):.3f}')
    print(f'Poly MSE: {cv_poly_MSE}')
    print(f'Poly MSE: {np.mean(cv_poly_MSE):.3f} +- {np.std(cv_poly_MSE):.3f}\n')
 
#     print(f'R^2 Ridge: {cv_ridge_r2}')
#     print(f'Poly ridge mean cv R^2: {np.mean(cv_ridge_r2):.3f} +- {np.std(cv_ridge_r2):.3f}')
#     print(f'Poly ridge MSE: {cv_ridge_MSE}')
#     print(f'Poly MSE: {np.mean(cv_ridge_MSE):.3f} +- {np.std(cv_ridge_MSE):.3f}\n')

    print(f'R^2 LASSO: {cv_LASSO_r2}')
    print(f'Poly LASSO mean cv R^2: {np.mean(cv_LASSO_r2):.3f} +- {np.std(cv_LASSO_r2):.3f}')
    print(f'Poly LASSO MSE: {cv_LASSO_MSE}')
    print(f'Poly MSE: {np.mean(cv_LASSO_MSE):.3f} +- {np.std(cv_LASSO_MSE):.3f}') 

In [ ]:
poly_add(X,y,.001)

In [ ]:
def Lasso_with_CV(X,y):
    lasso_model_score = []    
    # Polynomial factors
    
    X['PY/G^2'] = X['Pass_Yds/gm'] ** 3 + X['Pass_Yds/gm'] ** 2
    X['Cmp/G^2'] = X['Cmp/gm'] ** 3 + X['Cmp/gm'] ** 2
    X['TD/gm^2'] = X['TD/gm'] ** 3 + X['TD/gm'] ** 2
    X['Yrs Xp^2'] = X['Yrs Xp']**2
    
    # Add interaction terms
    X['TD%_/_Int%'] = X['TD%'] / X['Int%']
    
    X_train, X_val, y_train, y_val = train_test_split(X, y)
    
    scaler = StandardScaler()
    scaler.fit(X_train.values)
    
    X_tr_scaled = scaler.transform(X_train.values)
    X_val_scaled = scaler.transform(X_val.values)
    
    alphavec = 10**np.linspace(-2,2,200)
    lasso_model = LassoCV(alphas=alphavec, cv=4)
    lasso_model.fit(X_tr_scaled, y_train)
    lasso_model_score.append(lasso_model.score(X_val_scaled, y_val))
    val_pred = lasso_model.predict(X_val_scaled)
    print(lasso_model.alpha_)
    print(MSE_calc(y_val, val_pred))
    print(lasso_model_score)
    

In [ ]:
Lasso_with_CV(X,y)

In [ ]:
def Ridge_with_CV(X,y):
    Ridge_model_score = []  
    # Polynomial factors
    X['PY/G^2'] = X['Pass_Yds/gm'] ** 3 + X['Pass_Yds/gm'] ** 2
    X['Cmp/G^2'] = X['Cmp/gm'] ** 3 + X['Cmp/gm'] ** 2
    X['TD/gm^2'] = X['TD/gm'] ** 3 + X['TD/gm'] ** 2
    
    # Add interaction terms
    X['TD%_/_Int%'] = X['TD%'] / X['Int%']
    
    # split, scale, transform
    X_train, X_val, y_train, y_val = train_test_split(X, y)
    scaler = StandardScaler()
    scaler.fit(X_train.values)
    X_tr_scaled = scaler.transform(X_train.values)
    X_val_scaled = scaler.transform(X_val.values)
    
    alphavec = 10**np.linspace(-2,2,200)
    ridge_model = RidgeCV(alphas=alphavec, cv=4)
    ridge_model.fit(X_tr_scaled, y_train)
    val_pred = ridge_model.predict(X_val_scaled)
    Ridge_model_score.append(ridge_model.score(X_val_scaled, y_val))
    print(ridge_model.alpha_)
    print(MSE_calc(y_val, val_pred))
    print(ridge_model.score)
    

In [ ]:
Ridge_with_CV(X,y)

# Testing the model on validation (2017) to get a feel

In [ ]:
X.isna().sum()

In [ ]:
y.isna().sum()

In [ ]:
test_input = data_stats.copy()

In [ ]:
add_deviation_feature(test_input, 'TD/gm', 'tier' )

In [ ]:
def test_model(X_train, y, X_test, alpha=0.01):
    
    X_train['PY/G^2'] = X_train['Pass_Yds/gm'] ** 3 + X_train['Pass_Yds/gm'] ** 2
    X_train['Cmp/G^2'] = X_train['Cmp/gm'] ** 3 + X_train['Cmp/gm'] ** 2
    X_train['TD/gm^2'] = X_train['TD/gm'] ** 3 + X_train['TD/gm'] ** 2
    X_train['Yrs Xp^2'] = X_train['Yrs Xp']**2    
    X_train['TD%_/_Int%'] = X_train['TD%'] / X_train['Int%']

    X_test['PY/G^2'] = X_test['Pass_Yds/gm'] ** 3 + X_test['Pass_Yds/gm'] ** 2
    X_test['Cmp/G^2'] = X_test['Cmp/gm'] ** 3 + X_test['Cmp/gm'] ** 2
    X_test['TD/gm^2'] = X_test['TD/gm'] ** 3 + X_test['TD/gm'] ** 2
    X_test['Yrs Xp^2'] = X_test['Yrs Xp']**2
    X_test['TD%_/_Int%'] = X_test['TD%'] / X_test['Int%']
    
    scaler = StandardScaler()
    X_tr_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
#     poly_model = LinearRegression()
#     poly_model.fit(X_train, y)
#     predictions = poly_model.predict(X_test)
    
    # LASSO
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_tr_scaled, y)
    ln_predictions = lasso_model.predict(X_test_scaled)
    coefs = dict(zip(X_test.columns,lasso_model.coef_))
    for key, val in coefs.items():
        print(key,val)
    return ln_predictions

In [ ]:
sel_features = ['name','Year', 'FY_TD', 'G', 'Career W %', 'Cmp/gm', 'Att/gm', 'TD/gm', 'TD%',
       'Int%', 'Pass_Yds/gm', 'Int/gm', 'Sk/gm', 'Yrs Xp', 'tier', 'TD/gm_Dev_tier']

In [ ]:
# need to filter features
test_input = test_input[sel_features]

In [ ]:
test_input = test_input[test_input['Year'] == 2017]

In [ ]:
test_input

In [ ]:
test_model = test_model(X,y,test_input.drop(['name','FY_TD'], axis=1))

In [ ]:
test_model

In [ ]:
test_predict=test_input[['name','FY_TD']]

In [ ]:
test_predict['Pred'] = np.exp(test_model)

In [ ]:
test_predict

In [ ]:
test_predict['residual'] = abs(test_predict['FY_TD'] - test_predict['Pred'])

In [ ]:
np.sum(test_predict['residual']) / len(test_predict['residual'])

# Loading the test data and running predictions on test set

In [ ]:
qb_set_2018 = test_df_2018.append(benched_injured_df)

In [ ]:
add_deviation_feature(qb_set_2018, 'TD/gm', 'tier' )

In [ ]:
qb_set_2018 = qb_set_2018[sel_features]

In [ ]:
def final_model(X_train, y, X_test, alpha=0.01):
    
    X_train['PY/G^2'] = X_train['Pass_Yds/gm'] ** 3 + X_train['Pass_Yds/gm'] ** 2
    X_train['Cmp/G^2'] = X_train['Cmp/gm'] ** 3 + X_train['Cmp/gm'] ** 2
    X_train['TD/gm^2'] = X_train['TD/gm'] ** 3 + X_train['TD/gm'] ** 2
    X_train['Yrs Xp^2'] = X_train['Yrs Xp']**2    
    X_train['TD%_/_Int%'] = X_train['TD%'] / X_train['Int%']

    X_test['PY/G^2'] = X_test['Pass_Yds/gm'] ** 3 + X_test['Pass_Yds/gm'] ** 2
    X_test['Cmp/G^2'] = X_test['Cmp/gm'] ** 3 + X_test['Cmp/gm'] ** 2
    X_test['TD/gm^2'] = X_test['TD/gm'] ** 3 + X_test['TD/gm'] ** 2
    X_test['Yrs Xp^2'] = X_test['Yrs Xp']**2
    X_test['TD%_/_Int%'] = X_test['TD%'] / X_test['Int%']
    
    scaler = StandardScaler()
    X_tr_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # LASSO
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_tr_scaled, y)
    ln_predictions = lasso_model.predict(X_test_scaled)
    print(lasso_model.coef_)
    
    return ln_predictions

In [ ]:
test_results = final_model(X,y,qb_set_2018.drop(['name','FY_TD'], axis=1))

In [ ]:
results_df = qb_set_2018[['name','FY_TD']]

In [ ]:
results_df['Pred'] = np.round_(np.exp(test_results), 0)

In [ ]:
results_df['Residual'] = results_df['Pred'] - results_df['FY_TD']

In [ ]:
results_df['% var'] = (results_df['Residual'] / results_df['FY_TD'])

In [ ]:
results_df.sort_values(by='FY_TD', ascending=False)

In [ ]:
plt.figure(figsize=(14,10))
plt.scatter(results_df['FY_TD'], results_df['Pred'])
plt.title('Actual vs. Projected Passing TD', fontsize=40)
plt.xlabel('Actual', fontsize=14)
plt.ylabel('Projected', fontsize=14)
plt.savefig('result_graph.png')

In [ ]:
results_df['Residual'].apply(lambda x: x**2).sum() / len(results_df['Residual'])

# And for my 2019 Predictions

In [ ]:
qb_df_raw_2019 = pd.read_pickle('2019_data_for_pred.pkl')

In [ ]:
# start_list_2019 = ['Josh Allen', 'Ryan Fitzpatrick', 'Cam Newton', 'Sam Darnold', 'Lamar Jackson',
#                    'Baker Mayfield', 'Ben Roethlisberger', 'Deshaun Watson', 'Philip Rivers', 'Ryan Tannehill',
#                    'Patrick Mahomes', 'Derek Carr', 'Tyrod Taylor', 'Dak Prescott', 'Carson Wentz', 'Dwayne Haskins',
#                    'Mitchell Trubisky', 'Matthew Stafford', 'Aaron Rodgers', 'Kirk Cousins', 'Matt Ryan', 'Drew Brees',
#                    'Tom Brady', 'Kyler Murray', 'Jared Goff', 'Jimmy Garoppolo', 'Russell Wilson']

In [ ]:
qb_df_raw_2019['TD%'] = qb_df_raw_2019.apply(lambda x: x['TD'] / x['Att'], axis=1)
qb_df_raw_2019['Int%'] = qb_df_raw_2019.apply(lambda x: x['Int'] / x['Att'], axis=1)

In [ ]:
qb_2019_data = career_ratings.merge(qb_df_raw_2019, on='name', how='right')

In [ ]:
# Manually filling in data for Jimmy G and Lamar
qb_2019_data['YTD_Rating'] = qb_2019_data.apply(calc_passer_rating, axis=1)
qb_2019_data.loc[21,'tier'] = 1
qb_2019_data.loc[22,'tier'] = 1

In [ ]:
add_deviation_feature(qb_2019_data, 'TD/gm', 'tier')

In [ ]:
qb_set_2019 = qb_2019_data[sel_features]

In [ ]:
qb_set_2019.head()

In [ ]:
qb_set_2019.drop(['name','FY_TD'], axis=1).head()

In [ ]:
predict_2019 = final_model(X,y,qb_set_2019.drop(['name','FY_TD'], axis=1))

In [ ]:
results_2019 = pd.DataFrame(qb_set_2019['name'])

In [ ]:
results_2019['2020 Prediction'] = np.round_(np.exp(predict_2019), 0)

In [ ]:
results_2019.sort_values(by='2020 Prediction',ascending=False)

# Compare the results and figure out confidence interval

In [ ]:
def get_prediction_interval(prediction, y_test, test_predictions, pi=.95):
    '''
    Get a prediction interval for a linear regression.
    
    INPUTS: 
        - Single prediction, 
        - y_test
        - All test set predictions,
        - Prediction interval threshold (default = .95) 
    OUTPUT: 
        - Prediction interval for single prediction
    '''
    
    #get standard deviation of y_test
    sum_errs = np.sum((y_test - test_predictions)**2)
    stdev = np.sqrt(1 / (len(y_test) - 2) * sum_errs)
#get interval from standard deviation
    one_minus_pi = 1 - pi
    ppf_lookup = 1 - (one_minus_pi / 2)
    z_score = stats.norm.ppf(ppf_lookup)
    interval = z_score * stdev
#generate prediction interval lower and upper bound
    lower, upper = prediction - interval, prediction + interval
    return lower, prediction, upper
get_prediction_interval(predictions[0], y_test, predictions)
OUTPUT: (19.24072024369257, 28.996723619824934, 38.752726995957296

In [ ]:
def lasso_test(X_train, y, X_test, alpha=0.01):
    
    X_train['PY/G^2'] = X_train['Pass_Yds/gm'] ** 3 + X_train['Pass_Yds/gm'] ** 2
    X_train['Cmp/G^2'] = X_train['Cmp/gm'] ** 3 + X_train['Cmp/gm'] ** 2
    X_train['TD/gm^2'] = X_train['TD/gm'] ** 3 + X_train['TD/gm'] ** 2
    X_train['TD%_/_Int%'] = X_train['TD%'] / X_train['Int%']

    X_test['PY/G^2'] = X_test['Pass_Yds/gm'] ** 3 + X_test['Pass_Yds/gm'] ** 2
    X_test['Cmp/G^2'] = X_test['Cmp/gm'] ** 3 + X_test['Cmp/gm'] ** 2
    X_test['TD/gm^2'] = X_test['TD/gm'] ** 3 + X_test['TD/gm'] ** 2
    X_test['TD%_/_Int%'] = X_test['TD%'] / X_test['Int%']
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
 
    # LASSO
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_scaled, y)
    predictions = lasso_model.predict(X_test_scaled)
    print(lasso_model.coef_)
    return predictions